In [6]:
from pysparkling import *
import h2o
hc = H2OContext.getOrCreate(spark)

Connecting to H2O server at http://192.168.50.223:54321... successful.


H2O cluster uptime:,8 mins 36 secs
H2O cluster timezone:,Asia/Shanghai
H2O data parsing timezone:,UTC
H2O cluster version:,3.20.0.7
H2O cluster version age:,18 days
H2O cluster name:,sparkling-water-xingyuwu_local-1537343014506
H2O cluster total nodes:,1
H2O cluster free memory:,723 Mb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"



Sparkling Water Context:
 * H2O name: sparkling-water-xingyuwu_local-1537343014506
 * cluster size: 1
 * list of used nodes:
  (executorId, host, port)
  ------------------------
  (driver,192.168.50.223,54321)
  ------------------------

  Open H2O Flow in browser: http://192.168.50.223:54321 (CMD + click in Mac OSX)

    


In [8]:
import pandas as pd
from pyspark import SQLContext

In [12]:
sql_ctx = SQLContext(sc)

In [13]:
df = pd.read_csv("iris2.csv")
sdf = sql_ctx.createDataFrame(df)

In [14]:
sdf.show()

+------------+-----------+------------+-----------+-----+
|petal_length|petal_width|sepal_length|sepal_width|class|
+------------+-----------+------------+-----------+-----+
|         5.1|        3.5|         1.4|        0.2|    a|
|         4.9|        3.0|         1.4|        0.2|    a|
|         4.7|        3.2|         1.3|        0.2|    a|
|         4.6|        3.1|         1.5|        0.2|    a|
|         5.0|        3.6|         1.4|        0.2|    a|
|         5.4|        3.9|         1.7|        0.4|    a|
|         4.6|        3.4|         1.4|        0.3|    a|
|         5.0|        3.4|         1.5|        0.2|    a|
|         4.4|        2.9|         1.4|        0.2|    a|
|         4.9|        3.1|         1.5|        0.1|    a|
|         5.4|        3.7|         1.5|        0.2|    a|
|         4.8|        3.4|         1.6|        0.2|    a|
|         4.8|        3.0|         1.4|        0.1|    a|
|         4.3|        3.0|         1.1|        0.1|    a|
|         5.8|

In [21]:
# convert spark dataframe to H2OFrame
hdf = hc.as_h2o_frame(sdf)
hdf["class"]= hdf["class"].asfactor()

In [22]:
hdf.describe()

Rows:150
Cols:5




,petal_length,petal_width,sepal_length,sepal_width,class
type,real,real,real,real,enum
mins,4.3,2.0,1.0,0.1,
mean,5.84333333333,3.054,3.75866666667,1.19866666667,
maxs,7.9,4.4,6.9,2.5,
sigma,0.828066127978,0.433594311362,1.76442041995,0.763160741701,
zeros,0,0,0,0,
missing,0,0,0,0,0
0,5.1,3.5,1.4,0.2,a
1,4.9,3.0,1.4,0.2,a
2,4.7,3.2,1.3,0.2,a


In [23]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [24]:
gbm_classifier = H2OGradientBoostingEstimator(
    distribution=" multinomial", ntrees=10, max_depth=3, min_rows=2, learn_rate=0.2)

In [25]:
gbm_classifier.train(x=range(0,hdf.ncol-1), y=hdf.ncol-1,
         training_frame=hdf)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [26]:
gbm_classifier

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_model_python_1537343030888_2


ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 0.00976685303214
RMSE: 0.0988273900907
LogLoss: 0.0782480973696
Mean Per-Class Error: 0.00666666666667
Confusion Matrix: Row labels: Actual class; Column labels: Predicted class



a,b,c,Error,Rate
50.0,0.0,0.0,0.0,0 / 50
0.0,49.0,1.0,0.02,1 / 50
0.0,0.0,50.0,0.0,0 / 50
50.0,49.0,51.0,0.0066667,1 / 150


Top-3 Hit Ratios: 


k,hit_ratio
1,0.9933333
2,1.0
3,1.0


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error
,2018-09-19 16:19:08,0.030 sec,0.0,0.6666667,1.0986123,0.64
,2018-09-19 16:19:08,0.228 sec,1.0,0.5313437,0.7584115,0.0266667
,2018-09-19 16:19:08,0.278 sec,2.0,0.4233365,0.5505065,0.0266667
,2018-09-19 16:19:08,0.314 sec,3.0,0.3390480,0.4121733,0.0266667
,2018-09-19 16:19:08,0.346 sec,4.0,0.2728966,0.3135394,0.02
,2018-09-19 16:19:08,0.379 sec,5.0,0.2232307,0.2435139,0.02
,2018-09-19 16:19:08,0.409 sec,6.0,0.1846307,0.1909104,0.0066667
,2018-09-19 16:19:08,0.427 sec,7.0,0.1552645,0.1513943,0.0066667
,2018-09-19 16:19:08,0.445 sec,8.0,0.1326877,0.1208821,0.0066667
,2018-09-19 16:19:08,0.459 sec,9.0,0.1144659,0.0975897,0.0066667


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
sepal_width,192.7606506,1.0,0.7743739
sepal_length,54.0381165,0.2803379,0.2170864
petal_length,1.3527106,0.0070176,0.0054342
petal_width,0.7730322,0.0040103,0.0031055
